In [1]:
import numpy as np
import pandas as pd

In [222]:
def BrownRobinson(A, max_iter=1000, eps=1e-3, seed=None):
    assert len(A.shape) == 2
    N = A.shape[0]
    M = A.shape[1]

    rng = np.random.default_rng(seed)

    i = rng.integers(low=0, high=N-1)
    A_cumsum = np.zeros(N)
    B_cumsum = np.zeros(M)
    v_hat = np.inf

    table_data = []
    for k in range(1, max_iter+1):
        table_row = []
        table_row.append(i+1)

        B_cumsum = B_cumsum + np.squeeze(A[i, :])
        j = rng.choice(np.flatnonzero(B_cumsum == B_cumsum.min()))
        table_row.extend(np.round(B_cumsum, 1))
        v_lower = B_cumsum[j] / k

        table_row.append(j+1)

        A_cumsum = A_cumsum + np.squeeze(A[:, j])
        i = rng.choice(np.flatnonzero(A_cumsum == A_cumsum.max()))
        table_row.extend(np.round(A_cumsum, 1))
        v_upper = A_cumsum[i] / k

        v_hat_new = ( v_lower + v_upper ) / 2
        table_row.extend(np.round([v_lower, v_hat_new, v_upper], 2))

        table_data.append(table_row)
        if abs(v_hat_new - v_hat) <= eps:
            break
        v_hat = v_hat_new

    # generate df
    A_labels = [ f'A_{i}' for i in range(1, N+1) ] 
    B_labels = [ f'B_{i}' for i in range(1, M+1) ] 
    v_labels = ['v_lower', 'v_hat', 'v_upper']
    df_table = pd.DataFrame(table_data,
                            columns=['i', *B_labels, 'j', *A_labels, *v_labels])
    df_table.index += df_table.index + 1

    return df_table

In [223]:
A = np.array([
    [7, 2, 9],
    [2, 9, 0],
    [9, 0, 11]
    ])

In [224]:
seed = 2024
br_table = BrownRobinson(A, 2024)
br_table.tail()

,i,B_1,B_2,B_3,j,A_1,A_2,A_3,v_lower,v_hat,v_upper
283,3,720.0,694.0,732.0,2,721.0,699.0,721.0,4.89,4.98,5.08
285,3,729.0,694.0,743.0,2,723.0,708.0,721.0,4.85,4.95,5.06
287,1,736.0,696.0,752.0,2,725.0,717.0,721.0,4.83,4.93,5.03
289,1,743.0,698.0,761.0,2,727.0,726.0,721.0,4.81,4.91,5.01
291,1,750.0,700.0,770.0,2,729.0,735.0,721.0,4.79,4.91,5.03


In [225]:
N, M = 4, 5
low, high = 50, 500
A = np.random.randint(low, high, ( N, M ))
display(pd.DataFrame(A))

,0,1,2,3,4
0,226,160,96,175,275
1,310,358,123,236,216
2,369,135,118,458,367
3,432,232,404,326,447


In [226]:
br_table = BrownRobinson(A)
display(br_table.tail())

,i,B_1,B_2,B_3,B_4,B_5,j,A_1,A_2,A_3,A_4,v_lower,v_hat,v_upper
605,2,115196.0,86226.0,86417.0,87197.0,105932.0,2,42528.0,86619.0,39324.0,86292.0,284.57,285.22,285.87
607,2,115506.0,86584.0,86540.0,87433.0,106148.0,3,42624.0,86742.0,39442.0,86696.0,284.67,285.00,285.34
609,2,115816.0,86942.0,86663.0,87669.0,106364.0,3,42720.0,86865.0,39560.0,87100.0,284.14,284.86,285.57
611,4,116248.0,87174.0,87067.0,87995.0,106811.0,3,42816.0,86988.0,39678.0,87504.0,284.53,285.25,285.96
613,4,116680.0,87406.0,87471.0,88321.0,107258.0,2,42976.0,87346.0,39813.0,87736.0,284.71,285.25,285.79
